<a href="https://colab.research.google.com/github/mrinaltak/HateSpeechDetection/blob/main/OffensEval_baseline_subtask_a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
# Use Google Colab
use_colab = True
try:
    from google.colab import drive
    colab_available = True
except:
    colab_available = False

if use_colab and colab_available:
    drive.mount('/content/drive')

    # cd to the appropriate working directory under my Google Drive
    %cd '/content/drive/My Drive/685'

    !pip install -r requirements.txt

!pip install sentencepiece

from datasets import load_dataset

import sys
import os
import random
import shutil
import copy
import inspect


import numpy as np
import torch
import transformers
import datasets
import sklearn.metrics
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tqdm

from transformers import BertForSequenceClassification, AdamW, BertConfig, BertTokenizer

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")



import pandas as pd
train = pd.read_csv('olid_train_v2.csv',sep='\t')
train['subtask_a'] =train['subtask_a'].map({'OFF': 1, 'NOT': 0})
val = train.sample(frac=0.065,replace=False)
final = pd.concat([train, val, val]).drop_duplicates(keep=False)

#sanity checks
assert len(final) == 13240 - len(val)
s1 = pd.merge(val, final, how='inner', on=['id'])
assert len(s1)==0

s1 = pd.merge(final, train, how='inner', on=['id'])
assert len(s1) == len(final)

s1 = pd.merge(val, train, how='inner', on=['id'])
assert len(s1) == len(val)

val.to_csv('olid_val_v2.csv',sep='\t')
final.to_csv('olid_train_v3.csv',sep='\t')

train_dataset = load_dataset('csv', data_files={'train': 'olid_train_v3.csv','valid': 'olid_val_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_a_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_a.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_b_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_b.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_c_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_c.csv'}, delimiter="\t", cache_dir='./olid_cache')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/685
  Using cached en_core_web_sm-2.3.1-py3-none-any.whl
Found device: Tesla K80, n_gpu: 1


Using custom data configuration default-89bb6838b430151c


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-89bb6838b430151c/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


...

  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-4c126f2099f7d8bf
Reusing dataset csv (./olid_cache/csv/default-4c126f2099f7d8bf/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-faf03cde57e11c2d
Reusing dataset csv (./olid_cache/csv/default-faf03cde57e11c2d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:


def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
set_seed(685)

train_dataloader = torch.utils.data.DataLoader(train_dataset['train'], shuffle=True, batch_size=8)
val_dataloader = torch.utils.data.DataLoader(train_dataset['valid'], shuffle=True, batch_size=8)
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], shuffle=True, batch_size=8)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    cache_dir='./bert_cache'
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tell pytorch to run this model on the GPU.
model.cuda()

batch_size = 99
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8
                )
epochs = 5

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:

import numpy as np
# function to get validation accuracy
def get_validation_performance(val_dataloader):
    # Put the model in evaluation mode
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    

    total_correct = 0

    for batch in val_dataloader:
      if len(batch) == 0: continue
      inputs = tokenizer(batch['tweet'],padding='longest',return_tensors="pt").to(device)
      labels = batch['subtask_a'].to(device)
      
      # Tell pytorch not to bother with constructing the compute graph during
      # the forward pass, since this is only needed for backprop (training).
      with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        outputs = model(**inputs)
        loss = outputs.loss
        logits = outputs.logits
        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = labels.to('cpu').numpy()

        # Calculate the number of correctly labeled examples in batch
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        num_correct = np.sum(pred_flat == labels_flat)
        total_correct += num_correct
        
    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_correct / (len(val_dataloader) * val_dataloader.batch_size)
    return avg_val_accuracy

In [ ]:


import random

# training loop

# For each epoch...
for epoch_i in range(0, epochs):
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...

    for batch in tqdm.notebook.tqdm(train_dataloader):
      if len(batch) == 0: continue

      inputs = tokenizer(batch['tweet'],padding='longest',return_tensors="pt").to(device)
      labels = batch['subtask_a'].to(device)
      # Clear the previously calculated gradient
      model.zero_grad()        

      # Perform a forward pass (evaluate the model on this training batch).
      outputs = model(**inputs, labels=labels)
      loss = outputs.loss
      logits = outputs.logits

      total_train_loss += loss.item()

      # Perform a backward pass to calculate the gradients.
      loss.backward()

      # Update parameters and take a step using the computed gradient.
      optimizer.step()
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set. Implement this function in the cell above.
    print(f"Total loss: {total_train_loss}")
    val_acc = get_validation_performance(val_dataloader)
    print(f"Validation accuracy: {val_acc}")
    torch.save(model.state_dict(),os.path.join('./trained_model_offenseval_baseline_subtask_a/', 'subtask_a_baseline_{}.pt'.format(epoch_i)))
    
print("")
print("Training complete!")

#get_validation_performance(test_dataloader)


======== Epoch 1 / 5 ========
Training...


  0%|          | 0/1548 [00:00<?, ?it/s]

Total loss: 750.4608678929508
Validation accuracy: 0.8032407407407407

======== Epoch 2 / 5 ========
Training...


  0%|          | 0/1548 [00:00<?, ?it/s]

Total loss: 608.0810922253877
Validation accuracy: 0.7662037037037037

======== Epoch 3 / 5 ========
Training...


...

Total loss: 435.60440730210394
Validation accuracy: 0.7962962962962963

======== Epoch 4 / 5 ========
Training...


  0%|          | 0/1548 [00:00<?, ?it/s]

Total loss: 286.9684353163466
Validation accuracy: 0.7824074074074074

======== Epoch 5 / 5 ========
Training...


  0%|          | 0/1548 [00:00<?, ?it/s]

Total loss: 188.87035529932473
Validation accuracy: 0.7824074074074074

Training complete!


In [61]:
import pandas as pd
test = pd.read_csv('olid_test_subtask_a.csv',sep='\t')

test['subtask_a'] =test['label'].map({'OFF': 1, 'NOT': 0})
test.to_csv('olid_test_subtask_a_v2.csv',sep='\t')
test_a_dataset = load_dataset('csv', data_files={'test': 'olid_test_subtask_a_v2.csv'}, delimiter="\t", cache_dir='./olid_cache')
test_dataloader = torch.utils.data.DataLoader(test_a_dataset['test'], shuffle=True, batch_size=8)

Using custom data configuration default-ed1109be0744c2e4


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to ./olid_cache/csv/default-ed1109be0744c2e4/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
get_validation_performance(test_dataloader)

0.8240740740740741